In [1]:
import requests
import json

In [2]:

def obtener_custom_attributes(url):
    try:
        # Realiza la solicitud GET
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Lista para almacenar los valores de "custom_attributes"
        custom_attributes_list = []

        # Recorre las variantes y busca en attributesRaw
        for variant in data.get("allVariants", []):
            attributes = variant.get("attributesRaw", [])
            
            for attribute in attributes:
                if attribute.get("name") == "custom_attributes":
                    custom_attributes_list.append(attribute.get("value", {}))  # Obtiene el valor

        # Guardar en un archivo JSON si hay resultados
        if custom_attributes_list:
            with open('custom_attributes.json', 'w', encoding='utf-8') as f:
                json.dump(custom_attributes_list, f, ensure_ascii=False, indent=4)
            print("custom_attributes guardado en 'custom_attributes.json'")
        else:
            print("No se encontró 'custom_attributes' en los datos.")

        return custom_attributes_list
    
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud: {e}")
    except json.JSONDecodeError as e:
        print(f"Error al decodificar JSON: {e}")

# URL de ejemplo
url = 'https://storage.googleapis.com/resources-prod-shelftia/scrapers-prueba/product.json'
resultado = obtener_custom_attributes(url)
print(resultado)


custom_attributes guardado en 'custom_attributes.json'
[{'es-CR': '{"storage":{"value":{"code":"refrigerated","name":"Refrigerado"},"name":"Almacenamiento","type":"pim_catalog_simpleselect"},"country_of_origin":{"value":{"code":"united_states","name":"Estados Unidos"},"name":"País de origen","type":"pim_catalog_simpleselect"},"imported_national":{"value":{"code":"imported","name":"Importado"},"name":"Importado o Nacional","type":"pim_catalog_simpleselect"},"form":{"value":{"code":"block","name":"Bloque"},"name":"Forma","type":"pim_catalog_simpleselect"},"allergens":{"value":[{"code":"Milk","name":"Leche"}],"name":"Alérgenos","type":"pim_catalog_multiselect"},"sku":{"value":"29041","name":"SKU (Item Number)","type":"pim_catalog_identifier"},"show_online":{"value":true,"name":"Mostrar en línea","type":"pim_catalog_boolean"},"add_to_cart":{"value":true,"name":"Agregar al carrito en búsqueda/categoría","type":"pim_catalog_boolean"},"vegan":{"value":false,"name":"Vegano","type":"pim_catalog

In [3]:
import json

def procesar_custom_attributes(lista_custom_attributes):
    resultado_final = []

    for item in lista_custom_attributes:
        # Asegurarse de que el JSON está en formato correcto
        if isinstance(item, dict) and "es-CR" in item:
            try:
                nested_json = json.loads(item["es-CR"])  # Convertir de string JSON a dict
            except json.JSONDecodeError:
                continue  # Si falla la conversión, omitir el elemento

            # Extraer datos requeridos
            resultado = {
                "allergens": [allergen["name"] for allergen in nested_json.get("allergens", {}).get("value", [])],
                "sku": nested_json.get("sku", {}).get("value", ""),
                "vegan": nested_json.get("vegan", {}).get("value", False),
                "kosher": nested_json.get("kosher", {}).get("value", False),
                "organic": nested_json.get("organic", {}).get("value", False),
                "vegetarian": nested_json.get("vegetarian", {}).get("value", False),
                "gluten_free": nested_json.get("gluten_free", {}).get("value", False),
                "lactose_free": nested_json.get("lactose_free", {}).get("value", False),
                "package_quantity": nested_json.get("package_quantity", {}).get("value", 0),
                "unit_size": nested_json.get("unit_size", {}).get("value", ""),
                "net_weight": nested_json.get("net_weight", {}).get("value", "")
            }

            resultado_final.append(resultado)  # Agregar a la lista final

    return resultado_final




In [4]:
# Llamar a la función con la lista obtenida
resultado_limpio = procesar_custom_attributes(resultado)

# Imprimir el JSON final
print(json.dumps(resultado_limpio, indent=4, ensure_ascii=False))

[
    {
        "allergens": [
            "Leche"
        ],
        "sku": "29041",
        "vegan": false,
        "kosher": false,
        "organic": false,
        "vegetarian": true,
        "gluten_free": true,
        "lactose_free": false,
        "package_quantity": 1.0,
        "unit_size": "452.0000",
        "net_weight": "452.0000"
    }
]


In [5]:
import csv
import json


# Función para guardar en CSV
def guardar_en_csv(lista, nombre_archivo):
    if lista:
        # Abrimos el archivo CSV en modo escritura
        with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as archivo_csv:
            # Determinar las claves del primer elemento de la lista (para los encabezados)
            fieldnames = lista[0].keys()
            
            # Crear un objeto DictWriter con los encabezados
            writer = csv.DictWriter(archivo_csv, fieldnames=fieldnames)
            
            # Escribir los encabezados
            writer.writeheader()
            
            # Escribir las filas de datos
            writer.writerows(lista)
            
        print(f"Datos guardados correctamente en {nombre_archivo}")
    else:
        print("La lista está vacía, no se puede guardar el archivo CSV.")




In [6]:
# Guardar los datos de 'allVariants' en un archivo CSV
guardar_en_csv(resultado_limpio, "output-product.csv")

Datos guardados correctamente en output-product.csv
